### Demonstration of an evaluation based on the MCTSAgent

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'../../../')
import DeepCrazyhouse.src.runtime.Colorer
from DeepCrazyhouse.src.domain.agent.NeuralNetAPI import NeuralNetAPI
from DeepCrazyhouse.src.domain.agent.player.MCTSAgent import MCTSAgent
from DeepCrazyhouse.src.domain.agent.player.RawNetAgent import RawNetAgent
from DeepCrazyhouse.src.domain.crazyhouse.GameState import GameState
from time import time
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
net = NeuralNetAPI(ctx='gpu')
#net = NeuralNetAPI(ctx='gpu')

In [ ]:
batch_size = 8 #32
nb_playouts = 256 #2048 #512 #4096 #512 #4096 #2048 #256 #512 #256 #7 #16
cpuct = 1 #0.1 #0.2 #1 #3 #0.1 #1 #1 #5 #1 #1 #0.1 #0.5 #1 #0.25 #0.25 #0.5 #1 #0.2 #0.2 #0.1 #0.2 #0.2 #0.5 #1 #3 #.5 #0.1 #1 #3 #3
dirichlet_epsilon = 0.25
nb_workers = 64 #256 #128 #64 #128

In [ ]:
rawAgent = RawNetAgent(net)
#rawNet = GreedyAgent(rawAgent)

In [ ]:
#mctsAgent = MCTSAgent(net, virtual_loss=3, batch_size=batch_size, nb_workers=batch_size, cpuct=cpuct, nb_playouts=nb_playouts, dirichlet_epsilon=dirichlet_epsilon)

In [ ]:
CENTI_CPUCT = 100 #300 #100

In [ ]:
AGENT = MCTSAgent(net, cpuct=1, nb_playouts_empty_pockets=2048, nb_playouts_filled_pockets=4096,
                  cpuct_decay=0, cpuct_min=0, search_depth=40,
                  nb_playouts_update=256, max_search_time_s=900, dirichlet_alpha=0.2, dirichlet_epsilon=0.1)

In [ ]:
board = chess.variant.CrazyhouseBoard()

# game: https://lichess.org/5mcYjBhV

#board.push_uci('e2e4')
#board.push_uci('e7e6')
#fen = 'r1bQ1bnr/p1P2ppp/2p1k2b/2pNp3/2B1P3/2N5/PPPK1PPP/R6R/Pqn w - - 30 16'

#fen = 'r1b2bnr/p1k2ppp/8/2pNp3/4P3/8/PPP2PPP/R3KB1R/PQPnpqbn b KQ - 23 12'
#fen = 'r1b2bnr/p2k1ppp/8/2pNp3/4P3/8/PPP2PPP/R3KB1R/PQPnpqbn w KQ - 24 13'

#fen = 'r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p1B1/4P3/2NP1N2/PPP2PPP/R2QK2R/ b KQ - 11 6'
#fen = 'r1b2bnr/p1k2ppp/8/2ppp3/4P3/2N5/PPP2PPP/R3KB1R/QPnpqbn w KQ - 22 12'
#fen = 'r3qr1k/p4pp1/3b4/2p5/6b1/1N1BP3/PP1PpPP1/R1BQK2R[NPPnnppp] b KQ - 2 21'
#fen = 'r1bk1bnr/p4ppp/2p5/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/QPpqbn w KQ - 22 12'
#fen = 'r1bQ1bnr/p1P2ppp/2p1k3/2pNp3/2B1P3/2N5/PPPK1PPP/R6R/Pqbn b - - 29 15'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p1B1/4P3/2NP1N2/PPP2PPP/R2QK2R/ b KQ - 11 6'
#fen = 'r1b2rk1/ppp2ppp/2np1q2/1BbNp3/4P3/3P1N2/PPP2PPP/R2QK2R/Nb b KQ - 15 8'
#fen ='r1bq1rk1/p1p1Nppp/2pp4/2bNp3/4P3/3P1N2/PPP2PPP/R2QK2R/Nbb b KQ - 19 10'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p3/4P3/2NP1N2/PPP2PPP/R1BQ1RK1/ b - - 11 6'
#fen = 'r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11'
#fen = 'r1bk1bnr/p4ppp/2p5/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/QPpqbn w KQ - 22 12'
#fen = 'r1bq1rk1/p1p1Nppp/2pp4/2bNp3/4P3/3P1N2/PPP2PPP/R2QK2R/Nbb b KQ - 19 10'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p1B1/4P3/2NP1N2/PPP2PPP/R2QK2R/ b KQ - 11 6'
#fen = 'r1b2rk1/ppp2ppp/2np1q2/1BbNp3/4P3/3P1N2/PPP2PPP/R2QK2R/Nb b KQ - 15 8'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p3/4P3/2NP1N2/PPP2PPP/R1BQK2R/ w KQ - 10 6'
#fen = 'r4qk1/pp2bppp/2p5/3p4/8/P1N1PpPN/1PP3nP/R1B2bK1/pnrrpqb b - - 81 41'
#fen = 'r2q1rk1/pp2Pppp/2p2n2/3pQ3/N3p2b/P2bP1P1/1PPN3P/R1B1KBr1/Pn b Q - 57 29'
#fen = 'r2B1rk1/ppp2ppp/2np4/2bBp3/4P1b1/3P1N2/PPP2PPP/R2n1RK1/NQq w - - 20 11'
#fen = 'r2B1rk1/ppp2ppp/2np4/2bBp3/4P3/3P1b2/PPP2PPP/3R1RK1/NNQnq w - - 22 12'
#fen = 'r1b2rk1/ppp2ppp/2np1q2/1BbNp3/4P3/3P1N2/PPP2PPP/R2QK2R/Nb b KQ - 15 8'
#fen = 'r2B1rk1/ppp2ppp/2np4/2bBp3/4P3/3P3Q/PPP2PbP/3R1RK1/NNpnq w - - 5 13'
#fen = 'r2B1rk1/ppp2ppp/2np4/2bBp3/4P3/3P2p1/PPP2PQP/3R1RK1/BNNnq w - - 26 14'
#fen = 'r2B1rk1/ppp2ppp/2np1B2/3Bp3/4P3/3P2p1/PPP2bQP/3R1RK1/NNpnq w - - 28 15'
#fen = 'r1bQkbnr/p4ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QPNPbn b KQkq - 0 9'
#board.set_fen('r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11')
#board.set_fen('r3qrk1/p4ppp/3b4/2p5/6b1/1N1pP3/PP1PNPP1/RBBQK2R[NPnppp] b KQ - 0 19')
#fen = 'r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11'

#fen = 'r2q1rk1/pp2Pppp/2p2n2/3pQ3/N3p2b/P2bP1P1/1PPN3P/R1B1KBr1/Pn b Q - 3 29'
#fen = 'r3qrk1/p4ppp/3b4/2p5/6b1/1N2P3/PP1PpPP1/RBBQK2R/NPnnppp w KQ - 1 20'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p1B1/4P3/2NP1N2/PPP2PPP/R2QK2R/ b KQ - 11 6'
#fen = 'r3qr1k/p4pp1/3b4/2p5/6b1/1N1BP3/PP1PpPP1/R1BQK2R[NPPnnppp] b KQ - 2 21'
#board.set_fen('1rbQ1bnr/p1P2ppp/1Np1k3/2p1p3/2B1N3/P7/P1Pb1PPP/RK5R/NQPp b - - 39 20')
#board.set_fen('1rbQ1bnr/p1P2ppp/1Np5/2p1pk2/2B1N3/P7/P1Pb1PPP/RK5R/NQPp w - - 40 21')
#board.set_fen('r1bQ1bnr/p1P2ppp/2p1k3/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/pqbn w KQ - 26 14')
#board.set_fen('r1bk1bnr/p4ppp/2p5/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/QPpqbn w KQ - 22 12')
#fen = 'r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11'

#fen = 'r1b2rk1/ppp2ppp/2np3b/3Np1N1/4Pb1P/3P4/PPP2PP1/R2QKB1R/Qn b KQ - 5 12'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/1Bb1p1B1/4P3/2NP1N2/PPP2PPP/R2QK2R/ b KQ - 11 6'
#fen = 'r1b2rk1/ppp2ppp/2np3b/3Np1N1/4PbQP/3P4/PPP2PP1/R3KB1R/NQ b KQ - 5 13'
#fen = 'r4rk1/ppp2ppp/2np3b/3Np1N1/4PbbP/3P4/PPP2PP1/R3KB1R/NQq w KQ - 26 14'
#fen = 'r1bqkbnr/p4ppp/2pp4/2p1B3/4P3/2N5/PPP2PPP/R2QKB1R/PNPn b KQkq - 15 8'
#fen = 'r1b2rk1/pppp1ppp/8/4p1b1/2BnPb2/2NP2R1/PPPp1QPP/5K1R/NQn b - - 5 22'
#fen = 'r1b2rk1/ppp2ppp/8/3Bp1b1/3nPb2/2NP2R1/PPPp1QPP/5K1R/PNQn b - - 7 23'#
#fen = 'r1b1kbnr/ppp1pppp/2n5/3q4/3P4/8/PPP1NPPP/RNBQKB1R/Pp b KQkq - 1 4'
#fen = 'r1b2rk1/pppp1ppp/8/4p1b1/2BnP3/2NP2b1/PPPp1QPP/5K1R/NQrn w - - 5 23'
#fen = 'r1b4N/ppppk1pp/8/3Qp1b1/3nP3/2NP2b1/PPPp2PP/5K1R/RPbqrn w - - 15 28'

#fen = 'rnbqkb1r/ppp1pppp/5n2/3P4/8/8/PPPP1PPP/RNBQKBNR/P w KQkq - 1 3'
fen = 'r4rk1/ppp2pp1/3p1q1p/n1bPp3/2B1B1b1/3P1N2/PPP2PPP/R2Q1RK1[Nn] w - - 2 13'
#fen = 'rnb2rk1/p3bppp/2p5/3p2P1/4n3/8/PPPPBPPP/RNB1K1NR/QPPq w KQ - 0 11'

#fen = 'r1b1kbnr/ppp1pppp/2n5/3q4/3P4/8/PPP1NPPP/RNBQKB1R/Pp b KQkq - 1 4'

#fen = 'r1b1k2r/ppp2ppp/2n5/3np3/3P4/2PBP3/PpPB1PPP/1Q2K1NR/QNrb b Kkq - 27 14'
board.set_fen(fen)
#board.push_uci('b1h7')

state = GameState(board)
board

In [ ]:
len(list(state.get_legal_moves()))

In [ ]:
def plot_moves_with_prob(moves, probs, only_top_x=None):
    
    # revert the ordering afterwards
    idx_order = np.argsort(probs)[::-1]
    
    if only_top_x is not None and only_top_x < len(idx_order):
        idx_order = idx_order[:only_top_x]
    
    #moves_ordered = moves[range(len(moves))] #idx_order[::-1]]
    probs_ordered = [] #probs[idx_order]
    
    moves_ordered = []
    for idx in idx_order:
        probs_ordered.append(probs[idx])
        moves_ordered.append(moves[idx])
        
    plt.barh(range(len(probs_ordered)), probs_ordered)
    plt.yticks(range(len(moves_ordered)), moves_ordered)


In [ ]:
t_s = time()
value, legal_moves, p_vec_small = AGENT.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (t_s - time()))

In [ ]:
len(legal_moves)

In [ ]:
p_vec_small.shape

In [ ]:
t_p = np.array(
[0.00290707 ,0.00290707 ,0.00266456 ,0.00290707 ,0.00242206 ,0.00217955
 ,0.00290707 ,0.00290707 ,0.00242206 ,0.00508965 ,0.00242206 ,0.00290707
 ,0.00266456 ,0.00242206 ,0.00314958 ,0.00266456 ,0.00339209 ,0.00314958
 ,0.00290707 ,0.00314958 ,0.00290707 ,0.00314957 ,0.00314958 ,0.00436213
 ,0.00799973 ,0.03782833 ,0.04655865 ,0.42583359 ,0.06547433 ,0.0729921
 ,0.00290707 ,0.00533216 ,0.00266456 ,0.00242206 ,0.00266456 ,0.00266456
 ,0.00290707 ,0.00290707 ,0.00242206 ,0.00290707 ,0.00266456 ,0.00290707
 ,0.00314958 ,0.01357733 ,0.00508964 ,0.00363448 ,0.00411962 ,0.00290704
 ,0.00266456 ,0.00217955 ,0.00460463 ,0.00266456 ,0.00290707 ,0.00290707
 ,0.00484714 ,0.00314958 ,0.00266456 ,0.00242206 ,0.00339209 ,0.00242206
 ,0.00411962 ,0.00290707 ,0.00290707 ,0.00266456 ,0.00411962 ,0.00339209
 ,0.00339209 ,0.00242206 ,0.00314958 ,0.00266456 ,0.00533216 ,0.00339209
 ,0.0036346  ,0.00266456 ,0.0036346  ,0.00290707 ,0.0036346  ,0.00242206
 ,0.00314958 ,0.00266456 ,0.00339209 ,0.00314958 ,0.00387711 ,0.00314958
 ,0.00557467 ,0.00266456 ,0.00339209 ,0.00242206 ,0.00290707 ,0.00339209
 ,0.00339209 ,0.00290707 ,0.00314958 ,0.0036346  ,0.00339209 ,0.00290707
 ,0.0036346  ,0.00314958 ,0.00436213 ,0.00290707 ,0.00290707 ,0.00266456
 ,0.00339209 ,0.00436213 ,0.00242206 ,0.00242206 ,0.00290707 ,0.00266456
 ,0.00314958 ,0.00242206 ,0.00242206 ,0.00266456]
)

In [ ]:
from chess import Move

In [ ]:
t_mv = [Move.from_uci('h8g8'), Move.from_uci('h8f8'), Move.from_uci('e8f8'), Move.from_uci('e8d8'), Move.from_uci('e8e7'), Move.from_uci('e8d7'), Move.from_uci('c8d7'), Move.from_uci('c8e6'), Move.from_uci('c8f5'), Move.from_uci('c8g4'), Move.from_uci('c8h3'), Move.from_uci('a8b8'), Move.from_uci('c6d8'), Move.from_uci('c6b8'), Move.from_uci('c6e7'), Move.from_uci('c6a5'), Move.from_uci('c6d4'), Move.from_uci('c6b4'), Move.from_uci('d5e7'), Move.from_uci('d5f6'), Move.from_uci('d5b6'), Move.from_uci('d5f4'), Move.from_uci('d5b4'), Move.from_uci('d5e3'), Move.from_uci('d5c3'), Move.from_uci('e8g8'), Move.from_uci('e5d4'), Move.from_uci('h7h6'), Move.from_uci('g7g6'), Move.from_uci('f7f6'), Move.from_uci('b7b6'), Move.from_uci('a7a6'), Move.from_uci('e5e4'), Move.from_uci('h7h5'), Move.from_uci('g7g5'), Move.from_uci('f7f5'), Move.from_uci('b7b5'), Move.from_uci('a7a5'), Move.from_uci('B@a1'), Move.from_uci('R@a1'), Move.from_uci('B@c1'), Move.from_uci('R@c1'), Move.from_uci('B@d1'), Move.from_uci('R@d1'), Move.from_uci('B@f1'), Move.from_uci('R@f1'), Move.from_uci('B@e2'), Move.from_uci('R@e2'), Move.from_uci('B@a3'), Move.from_uci('R@a3'), Move.from_uci('B@b3'), Move.from_uci('R@b3'), Move.from_uci('B@f3'), Move.from_uci('R@f3'), Move.from_uci('B@g3'), Move.from_uci('R@g3'), Move.from_uci('B@h3'), Move.from_uci('R@h3'), Move.from_uci('B@a4'), Move.from_uci('R@a4'), Move.from_uci('B@b4'), Move.from_uci('R@b4'), Move.from_uci('B@c4'), Move.from_uci('R@c4'), Move.from_uci('B@e4'), Move.from_uci('R@e4'), Move.from_uci('B@f4'), Move.from_uci('R@f4'), Move.from_uci('B@g4'), Move.from_uci('R@g4'), Move.from_uci('B@h4'), Move.from_uci('R@h4'), Move.from_uci('B@a5'), Move.from_uci('R@a5'), Move.from_uci('B@b5'), Move.from_uci('R@b5'), Move.from_uci('B@c5'), Move.from_uci('R@c5'), Move.from_uci('B@f5'), Move.from_uci('R@f5'), Move.from_uci('B@g5'), Move.from_uci('R@g5'), Move.from_uci('B@h5'), Move.from_uci('R@h5'), Move.from_uci('B@a6'), Move.from_uci('R@a6'), Move.from_uci('B@b6'), Move.from_uci('R@b6'), Move.from_uci('B@d6'), Move.from_uci('R@d6'), Move.from_uci('B@e6'), Move.from_uci('R@e6'), Move.from_uci('B@f6'), Move.from_uci('R@f6'), Move.from_uci('B@g6'), Move.from_uci('R@g6'), Move.from_uci('B@h6'), Move.from_uci('R@h6'), Move.from_uci('B@d7'), Move.from_uci('R@d7'), Move.from_uci('B@e7'), Move.from_uci('R@e7'), Move.from_uci('B@b8'), Move.from_uci('R@b8'), Move.from_uci('B@d8'), Move.from_uci('R@d8'), Move.from_uci('B@f8'), Move.from_uci('R@f8'), Move.from_uci('B@g8'), Move.from_uci('R@g8')]

In [ ]:
len(t_mv)

In [ ]:
t_p.shape

In [ ]:
AGENT.get_calclated_line()

In [ ]:
AGENT.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
len(mctsAgent.get_calclated_line()[0])

In [ ]:
t_s = time()
value, selected_move, confidence, legal_moves, p_vec_small = mctsAgent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (t_s - time()))

In [ ]:
t_s = time()
value, selected_move, confidence, legal_moves, p_vec_small = mctsAgent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (t_s - time()))

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

## cpuct 1.0

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

## cpuct 0.75

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

## cpuct 3

In [ ]:
mctsAgent.get_calclated_line()

## cpuct 0.2

In [ ]:
mctsAgent.get_calclated_line()

## cpuct 0.1

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, mctsAgent.root_node.p, only_top_x=10)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
board = chess.variant.CrazyhouseBoard()
#board.push_uci('e2e4')
#board.set_fen('r1bQ1bnr/p1P2ppp/2p1k2b/2pNp3/2B1P3/2N5/PPPK1PPP/R6R/Pqn w - - 30 16')

fen = 'r1b2bnr/p1k2ppp/8/2pNp3/4P3/8/PPP2PPP/R3KB1R/PQPnpqbn b KQ - 23 12'
#fen = 'r1b2bnr/p2k1ppp/8/2pNp3/4P3/8/PPP2PPP/R3KB1R/PQPnpqbn w KQ - 24 13'
fen = 'r1b2bnr/p1k2ppp/8/2pNp3/4P3/8/PPP2PPP/R3KB1R[QPPqbnnp] b KQ - 23 12'
fen = 'r1b2bnr/p4ppp/3k4/2pNp3/4P3/8/PPP2PPP/R3KB1R/PQPnpqbn w KQ - 1 13'
#fen = 'r1b2bnr/p1k2ppp/8/2ppp3/4P3/2N5/PPP2PPP/R3KB1R/QPnpqbn w KQ - 22 12'
fen = 'r1b2bnr/p4ppp/3k4/2pNp3/4P3/8/PPP2PPP/R3KB1R[QPPqbnnp] w KQ - 1 13'
fen = 'r1b2bnr/p4ppp/3k4/2pNp3/4P3/8/PPP2PPP/R3KB1R[QPPqbnnp] w KQ - 1 13'

board.set_fen('r1b2bnr/p1k2ppp/2p5/2p1p3/4P3/2N5/PPP2PPP/R3KB1R/QNPpqbn w KQ - 20 11')
#board.set_fen('r3qrk1/p4ppp/3b4/2p5/6b1/1N1pP3/PP1PNPP1/RBBQK2R[NPnppp] b KQ - 0 19')

fen = 'r1b2rk1/pppp1ppp/8/4p1b1/2BnP3/2NP2b1/PPPp1QPP/5K1R/NQrn w - - 44 23'
#fen = 'r3qrk1/p4ppp/3b4/2p5/6b1/1N2P3/PP1PpPP1/RBBQK2R/NPnnppp w KQ - 1 20'
#fen = 'r3qr1k/p4pp1/3b4/2p5/6b1/1N1BP3/PP1PpPP1/R1BQK2R[NPPnnppp] b KQ - 2 21'
#board.set_fen('1rbQ1bnr/p1P2ppp/1Np1k3/2p1p3/2B1N3/P7/P1Pb1PPP/RK5R/NQPp b - - 39 20')
#board.set_fen('1rbQ1bnr/p1P2ppp/1Np5/2p1pk2/2B1N3/P7/P1Pb1PPP/RK5R/NQPp w - - 40 21')
#board.set_fen('r1bQ1bnr/p1P2ppp/2p1k3/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/pqbn w KQ - 26 14')
#board.set_fen('r1bk1bnr/p4ppp/2p5/2pNp3/4P3/2N5/PPP2PPP/R3KB1R/QPpqbn w KQ - 22 12')
board.set_fen(fen)
#board.push_uci('b1h7')

state = GameState(board)
board

In [ ]:
rawNet.perform_action(state)

In [ ]:
pred_value, selected_moves, move_probabilites = rawAgent.evaluate_board_state(state)

In [ ]:
plot_moves_with_prob(selected_moves, move_probabilites, only_top_x=10)

In [ ]:
import chess
import chess.variant

In [ ]:
b = chess.variant.CrazyhouseBoard('r1b2rk1/pppp1ppp/8/4p1b1/2BnP3/2NP2b1/PPPp1QPP/5K1R/NQrn w - - 44 23')
b

In [ ]:
from copy import deepcopy

In [ ]:
check_moves_enhance = np.zeros(len(list(b.legal_moves)))

for i, mv in enumerate(b.legal_moves):
    b_tmp = deepcopy(b)
    b_tmp.push(mv)
    if b_tmp.is_check():
        print(b_tmp.is_check(), mv)
        check_moves_enhance[i] = 1

In [ ]:
((check_moves_enhance / check_moves_enhance.sum()) * .5) / 1.5

In [ ]:
mctsAgent.root_node.p

In [ ]:
#plot_moves_with_prob(mctsAgent.root_node.legal_moves, mctsAgent.root_node.p)

In [ ]:
confidence

In [ ]:
p_vec_small

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
r = mctsAgent.root_node

In [ ]:
n = r.child_nodes[0]

In [ ]:
n.legal_moves

In [ ]:
n.q

In [ ]:
n.n

In [ ]:
mctsAgent.show_next_pred_line()

In [ ]:
n = n.child_nodes[2]

In [ ]:
n.q

In [ ]:
n.n

In [ ]:
n.q

In [ ]:
n.n

In [ ]:
n.p

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
def plot_move_dict(move_dict):
    moves_ordered = sorted(move_dict, key=move_dict.__getitem__)
    probs = sorted(move_dict.values())
    
    plt.barh(range(len(probs)), probs)
    plt.yticks(range(len(probs)), moves_ordered)

In [ ]:
plot_move_dict(move_dict)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
epsilon=0.25
alpha=0.15

In [ ]:
dirichlet_noise = epsilon * np.random.dirichlet([alpha] * 20)

In [ ]:
plt.hist(dirichlet_noise)

In [ ]:
mctsAgent.root_node.q

In [ ]:
mctsAgent.root_node.w 

In [ ]:
plot_move_dict(move_dict)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
plot_move_dict(move_dict)

In [ ]:
mctsAgent.get_calclated_line()

In [ ]:
mctsAgent.root_node.child_nodes

In [ ]:
mctsAgent.root_node.nb_direct_child_nodes

In [ ]:
mctsAgent.root_node.nb_total_expanded_child_nodes

In [ ]:
mctsAgent.root_node.nb_expandable_child_nodes

In [ ]:
mctsAgent.root_node.child_nodes[1].child_nodes